In [1]:
import json
import requests
import pyarrow.parquet as pq
import os
import openai
import random
label_map = {'exact': 3, 'substitute':2, 'complement':1, 'irrelevant':0}
label_map2 = {'E': 3, 'S':2, 'C':1, 'I':0}
openai.api_key = 'YOUROPENAIKEY'#os.getenv("OPENAI_API_KEY")


In [2]:
id2imgurl = {}
asin2trecid = {}
trecid2asin = {}
doc_id = 0
collection = {}
with open('esci.json','r') as f:
    for l in f:
        j = json.loads(l)
        trecid2asin [doc_id] = j['asin']
        asin2trecid[j['asin']] = doc_id
        try:
            id2imgurl[doc_id] = j['image']
        except:
            pass
        j['doc_id'] = doc_id
        collection[doc_id] = j
        if 'description' not in collection[doc_id]:
            collection[doc_id]['description'] = ''
        if 'title' not in collection[doc_id]:
            collection[doc_id]['title'] = ''
        collection[doc_id]['docid'] = collection[doc_id]['doc_id']
        collection[doc_id]['text'] = collection[doc_id]['description']
        doc_id += 1

In [3]:
collection[0]

{'type': 'product',
 'locale': 'es',
 'asin': 'B06XXK1JQN',
 'title': 'Estores Basic- Enrollable Traslúcido , Crudo, 150x250 cm',
 'stars': '3,9 de 5 estrellas',
 'ratings': '90 valoraciones',
 'category': ['Hogar y cocina',
  'Decoración del hogar',
  'Decoración de ventanas',
  'Persianas',
  'Persianas enrollables y estores'],
 'attrs': {'Número de productos': '1',
  'Material': 'Textura',
  'Función especial': 'sin cuerda',
  'Marca': 'Estores Basic',
  'Color': 'Crudo',
  'Tamaño': '150x6x250 cm',
  'Peso del producto': '2.76 Libras',
  'Tipo de montaje': 'Montaje interior'},
 'bullets': ['Estor enrollable Traslúcido con contrapeso de PVC, oculto',
  'Mecanismo de accionamiento de cadeneta',
  'Soportes metálicos con embellecedores para instalación a techo o pared',
  'Limpieza con un paño ligeramente húmedo',
  'Dimensiones del producto \u200f : \u200e 150 x 6 x 250 cm; 1.25 kilogramos',
  'Producto en Amazon.es desde \u200f : \u200e 28 marzo 2017',
  'Fabricante \u200f : \u200e 

In [4]:
print("There are {} documents in the collection".format(doc_id -1))
print("There are {} img urls in the collection".format(len(id2imgurl)))

There are 1661907 documents in the collection
There are 1480714 img urls in the collection


In [14]:
with open('collection-imgs.json','w') as w:
    for i in id2imgurl:
        j = {'doc_id': i, 'image_url':id2imgurl[i]}
        w.write('{}\n'.format(json.dumps(j)))

In [5]:
asin2trecid[collection[0]['asin']]

0

In [6]:
collection[doc_id-1]

{'type': 'product',
 'locale': 'us',
 'asin': 'B002VUAM46',
 'title': 'Degree Men Antiperspirant Deodorant Adventure Antiperspirant For Men With MotionSense Technology, 2.7 Ounce (Pack of 4)',
 'stars': '4.7 out of 5 stars',
 'ratings': '2,256 ratings',
 'category': ['Beauty & Personal Care',
  'Personal Care',
  'Deodorants & Antiperspirants',
  'Antiperspirants'],
 'attrs': {'Material Type Free': 'ERROR: #N/A',
  'Item Form': 'Sticks',
  'Brand': 'DEGREE',
  'Item Volume': '2.7 Fluid Ounces',
  'Scent': 'Woodsy'},
 'bullets': ['We are upgrading! For a limited time, you may receive our old 48-hour Antiperspirant Deodorant while all our new 72-hour products make it to stores. Detailed changes in product description',
  'MOTIONSENSE TECHNOLOGY: This clinically proven antiperspirant provides a unique motion-activated, odor-control technology with 2x stronger sweat protection than regular antiperspirants',
  'READY FOR WHAT’S NEXT: Take on the day with Degree Men Adventure Antiperspirant 

# Generate a complete dump of all dataset withouth any language specific filtering

In [8]:
with open('full-collection.jsonl','w') as w:
    for i in range(0,len(collection)):
        doc_id = asin2trecid[collection[i]['asin']]
        collection[i]['doc_id'] = doc_id
        w.write("{}\n".format(json.dumps(j)))

In [35]:
from pyarrow import json
table = json.read_json('full-collection.jsonl') 
pq.write_table(table, 'full-collection.parquet')
import json

In [89]:
in_english = set()
for i in range(0,len(collection)):
    if collection[i]['locale'] == 'us':
        in_english.add(i)
print(len(in_english))

1118658


## Generate a TREC collection style 

In [9]:
#doc_id\ttitle\tdescription
with open('collection.trec', 'w') as w:
    for i in range(0,len(collection)):
        if collection[i]['locale'] == 'us':
            w.write("{}\t{}\t{}\n".format(collection[i]['doc_id'],collection[i]['title'],collection[i]['description']))

## Generate a JSON collection style 

In [10]:
#json 
with open('corpus.jsonl', 'w') as w:
    for i in range(0,len(collection)):
        if collection[i]['locale'] == 'us':
            w.write("{}\n".format(json.dumps(collection[i])))

In [36]:
from pyarrow import json
table = json.read_json('corpus.jsonl') 
pq.write_table(table, 'corpus.parquet')
import json

## Pyserini Compatable

Next we go ahead and make a collection in the trec format. 

In [22]:
! mkdir indexes esci-pyserini-simple esci-pyserini-full

mkdir: indexes: File exists


In [24]:
# pyserini with just title and description
with open('esci-pyserini-simple/collection.jsonl','w') as w: 
    for i in range(0,len(collection)):
        if collection[i]['locale'] == 'us':
            j = {}
            j['id'] = collection[i]['doc_id']
            j['contents'] = json.dumps({'title':collection[i]['title'],'text':collection[i]['text']})
            w.write("{}\n".format(json.dumps(j)))

In [37]:
from pyarrow import json
table = json.read_json('esci-pyserini-simple/collection.jsonl') 
pq.write_table(table, 'esci-pyserini-simple/collection.parquet')
import json

In [25]:
# pyserini with just title and description
with open('esci-pyserini-full/collection.jsonl','w') as w: 
    for i in range(0,len(collection)):
        if collection[i]['locale'] == 'us':
            j = {}
            j['id'] = collection[i]['doc_id']
            j['contents'] = json.dumps(collection[i])
            w.write("{}\n".format(json.dumps(j)))

In [38]:
from pyarrow import json
table = json.read_json('esci-pyserini-full/collection.jsonl') 
pq.write_table(table, 'esci-pyserini-full/collection.parquet')
import json

In [27]:
! python -m pyserini.index.lucene --collection JsonCollection --input esci-pyserini-simple --index indexes/esci-simple-pyserini --generator DefaultLuceneDocumentGenerator --threads 1 --storePositions --storeDocvectors --storeRaw

2023-04-11 10:50:35,578 INFO  [main] index.IndexCollection (IndexCollection.java:645) - Setting log level to INFO
2023-04-11 10:50:35,580 INFO  [main] index.IndexCollection (IndexCollection.java:648) - Starting indexer...
2023-04-11 10:50:35,580 INFO  [main] index.IndexCollection (IndexCollection.java:649) - ============ Loading Parameters ============
2023-04-11 10:50:35,581 INFO  [main] index.IndexCollection (IndexCollection.java:650) - DocumentCollection path: esci-pyserini-simple
2023-04-11 10:50:35,581 INFO  [main] index.IndexCollection (IndexCollection.java:651) - CollectionClass: JsonCollection
2023-04-11 10:50:35,581 INFO  [main] index.IndexCollection (IndexCollection.java:652) - Generator: DefaultLuceneDocumentGenerator
2023-04-11 10:50:35,582 INFO  [main] index.IndexCollection (IndexCollection.java:653) - Threads: 1
2023-04-11 10:50:35,582 INFO  [main] index.IndexCollection (IndexCollection.java:654) - Language: en
2023-04-11 10:50:35,582 INFO  [main] index.IndexCollection (I

In [28]:
! python -m pyserini.index.lucene --collection JsonCollection --input esci-pyserini-full --index indexes/esci-full-pyserini --generator DefaultLuceneDocumentGenerator --threads 1 --storePositions --storeDocvectors --storeRaw

2023-04-11 11:56:42,186 INFO  [main] index.IndexCollection (IndexCollection.java:645) - Setting log level to INFO
2023-04-11 11:56:42,187 INFO  [main] index.IndexCollection (IndexCollection.java:648) - Starting indexer...
2023-04-11 11:56:42,187 INFO  [main] index.IndexCollection (IndexCollection.java:649) - ============ Loading Parameters ============
2023-04-11 11:56:42,188 INFO  [main] index.IndexCollection (IndexCollection.java:650) - DocumentCollection path: esci-pyserini-full
2023-04-11 11:56:42,188 INFO  [main] index.IndexCollection (IndexCollection.java:651) - CollectionClass: JsonCollection
2023-04-11 11:56:42,188 INFO  [main] index.IndexCollection (IndexCollection.java:652) - Generator: DefaultLuceneDocumentGenerator
2023-04-11 11:56:42,188 INFO  [main] index.IndexCollection (IndexCollection.java:653) - Threads: 1
2023-04-11 11:56:42,189 INFO  [main] index.IndexCollection (IndexCollection.java:654) - Language: en
2023-04-11 11:56:42,189 INFO  [main] index.IndexCollection (Ind

In [4]:
from pyserini.search.lucene import LuceneSearcher

searcher = LuceneSearcher('indexes/esci-full-pyserini')
hits = searcher.search('iphone chargers', 100)

for i in range(len(hits)):
   print(f'{i+1:2} {hits[i].docid:4} {hits[i].score:.5f}')

 1 1167339 6.89770
 2 350574 6.88980
 3 779772 6.88730
 4 1575716 6.88410
 5 1160154 6.88360
 6 691412 6.88030
 7 1643780 6.87890
 8 1201683 6.87530
 9 1360375 6.87390
10 1571386 6.87370
11 1265234 6.87350
12 966314 6.87270
13 1425213 6.87260
14 1593423 6.87130
15 605874 6.87060
16 787982 6.86960
17 999017 6.86930
18 743049 6.86860
19 112904 6.86680
20 600733 6.86560
21 367145 6.86530
22 1544772 6.86470
23 126680 6.86450
24 486218 6.86030
25 1264548 6.85830
26 1091510 6.85750
27 1580381 6.85650
28 161353 6.85630
29 771950 6.85360
30 268918 6.85300
31 1093691 6.85290
32 976335 6.85250
33 878434 6.85140
34 497237 6.85110
35 1173589 6.85070
36 940791 6.85060
37 1549372 6.84760
38 167246 6.84670
39 581918 6.84620
40 1129931 6.84600
41 1318119 6.84100
42 218634 6.84090
43 4701 6.83990
44 715097 6.83960
45 1243454 6.83930
46 349711 6.83730
47 667817 6.83680
48 336347 6.83590
49 33430 6.83500
50 368480 6.83360
51 1140007 6.83320
52 1292138 6.83090
53 919554 6.82970
54 1003394 6.82900
55 17697

In [17]:
collection[1167339]['title']

'iPhone Charger 90 Degree Lightning Cable 6ft 5 Pack Fast Charging Cable iPhone Charging Cords Right Angle USB Data Cable Compatible with iPhone 11/11 pro/Xs Max/XS/XR/7/7Plus/X/8/8Plus/6S/6S Plus/SE'

In [18]:
from pyserini.search.lucene import LuceneSearcher

searcher = LuceneSearcher('indexes/esci-simple-pyserini')
hits = searcher.search('iphone chargers', 100)

for i in range(len(hits)):
   print(f'{i+1:2} {hits[i].docid:4} {hits[i].score:.5f}')

 1 1167339 8.10870
 2 1575716 8.09710
 3 1160154 8.08380
 4 691412 8.08050
 5 743049 8.07740
 6 605874 8.05650
 7 787982 8.04570
 8 779772 8.03540
 9 1360375 8.03010
10 1643780 8.02680
11 112904 8.02360
12 1425213 8.01630
13 966314 8.01330
14 1571386 8.00700
15 497237 8.00290
16 878434 8.00290
17 350574 7.99980
18 1544772 7.97790
19 161353 7.97340
20 486218 7.96780
21 1091510 7.94770
22 1173589 7.94630
23 268918 7.94180
24 999017 7.92920
25 600733 7.92610
26 1549372 7.91820
27 581918 7.91180
28 33430 7.91090
29 126680 7.89990
30 1593423 7.87880
31 1292138 7.87830
32 940791 7.87700
33 1201683 7.87470
34 314262 7.85350
35 976335 7.84830
36 1265234 7.83090
37 176972 7.82490
38 349711 7.82400
39 167246 7.81940
40 1383863 7.81910
41 297667 7.81610
42 1093691 7.81480
43 679673 7.81380
44 1533463 7.80410
45 367145 7.79960
46 218634 7.79710
47 336347 7.79470
48 1264548 7.79300
49 971992 7.78910
50 1169113 7.78890
51 1381463 7.78770
52 1140007 7.78760
53 1318119 7.78650
54 715097 7.78650
55 113

In [12]:
! git clone https://github.com/amazon-science/esci-data.git

Cloning into 'esci-data'...
remote: Enumerating objects: 124, done.
remote: Counting objects: 100% (88/88), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 124 (delta 49), reused 48 (delta 24), pack-reused 36
Receiving objects: 100% (124/124), 390.33 KiB | 4.94 MiB/s, done.
Resolving deltas: 100% (59/59), done.
Filtering content: 100% (2/2), 1.08 GiB | 39.45 MiB/s, done.


In [13]:
! ls esci-data/shopping_queries_dataset

shopping_queries_dataset_examples.parquet
shopping_queries_dataset_products.parquet
shopping_queries_dataset_sources.csv


In [7]:
import pandas as pd
df_examples = pd.read_parquet('esci-data/shopping_queries_dataset/shopping_queries_dataset_examples.parquet')
df_products = pd.read_parquet('esci-data/shopping_queries_dataset/shopping_queries_dataset_products.parquet')
df_sources = pd.read_csv("esci-data/shopping_queries_dataset/shopping_queries_dataset_sources.csv")

In [8]:
df_examples_products = pd.merge(
    df_examples,
    df_products,
    how='left',
    left_on=['product_locale','product_id'],
    right_on=['product_locale', 'product_id']
)

In [9]:
df_task_1 = df_examples_products[df_examples_products["small_version"] == 1]
df_task_1_train = df_task_1[df_task_1["split"] == "train"]
df_task_1_test = df_task_1[df_task_1["split"] == "test"]

In [10]:
print("Train Labels {}".format(len(df_task_1_train)))
print("Dev Labels {}".format(len(df_task_1_test)))

Train Labels 781638
Dev Labels 336373


In [11]:
qid2query = {}
for row in df_task_1_train.iterrows():
    if row[1]['product_locale'] == 'us':
        qid = int(row[1]['query_id'])
        query_text = row[1]['query']
        qid2query[qid] = query_text

for row in df_task_1_test.iterrows():
    if row[1]['product_locale'] == 'us':
        qid = int(row[1]['query_id'])
        query_text = row[1]['query']
        qid2query[qid] = query_text
    
    
with open('qid2query.tsv','w') as w:
    for qid in qid2query:
        w.write("{}\t{}\n".format(qid, qid2query[qid]))
        
        
print(len(qid2query))

29844


In [12]:
train = {}
for row in df_task_1_train.iterrows():
    if row[1]['product_locale'] == 'us':
        try:
            qid = int(row[1]['query_id'])
            query_text = row[1]['query']
            label = label_map2[row[1]['esci_label']]
            if label >= 0:
                pid = row[1]['product_id']
                if qid not in train:
                    train[qid] = {}
                    train[qid]['query_text'] = query_text
                    train[qid]['labels'] = {}
                trec_id = asin2trecid[pid]
                train[qid]['labels'][trec_id] = label
        except:
            pass
print("There are {} queries in the train postion of the dataset".format(len(train)))
print("An example query and its judgements is:\n{}".format(train[1]))

There are 20888 queries in the train postion of the dataset
An example query and its judgements is:
{'query_text': '!awnmower tires without rims', 'labels': {1049092: 0, 314519: 3, 689593: 0, 717649: 2, 1397156: 3, 1095149: 3, 341130: 1, 911618: 3, 831070: 2, 1624211: 0, 116997: 2, 1399124: 3, 1035122: 3, 932695: 0, 580389: 0}}


In [105]:
with open('product-search-train.qrels','w') as w:
    for qid in train:
        for doc_id in train[qid]['labels']:
            relevancy = train[qid]['labels'][doc_id]
            w.write("{}\t0\t{}\t{}\n".format(qid, doc_id, relevancy))

In [106]:
with open('product-search-train-qrels.jsonl','w') as w:
    for qid in train:
        for doc_id in train[qid]['labels']:
            relevancy = train[qid]['labels'][doc_id]
            j = {'qid':qid, 'Q0':'Q0', 'docid':doc_id, 'rel':relevancy }
            w.write("{}\n".format(json.dumps(j)))

from pyarrow import json
table = json.read_json('product-search-train-qrels.jsonl') 
pq.write_table(table, 'product-search-train-qrels.parquet')
import json

In [80]:
df_task_1_test

,example_id,query,query_id,product_id,product_locale,esci_label,small_version,large_version,split,product_title,product_description,product_bullet_point,product_brand,product_color
32,32,!qscreen fence without holes,2,B07D7TBSGH,us,I,1,1,test,FOTMISHU 6Pcs Greenhouse Hoops Rust-Free Grow ...,<p><b>Description:</b><br><b>Material:</b>plas...,►Material -- High-quality plastic coated steel...,FOTMISHU,green
33,33,!qscreen fence without holes,2,B07DHT2WZK,us,I,1,1,test,Zippity Outdoor Products ZP19028 Unassembled M...,None,Designed for the Madison Vinyl Fence (item ZP1...,Zippity Outdoor Products,White
34,34,!qscreen fence without holes,2,B07DHX8YH2,us,E,1,1,test,Zippity Outdoor Products ZP19026 Lightweight P...,None,Designed as a portable fence for temporary eve...,Zippity Outdoor Products,White
35,35,!qscreen fence without holes,2,B07DS1YCRZ,us,S,1,1,test,ColourTree 4' x 50' Green Fence Privacy Screen...,None,"★【Virgin, UV Stabilized Compounds & Thick Mate...",ColourTree,Green 2nd Generation
36,36,!qscreen fence without holes,2,B07DS3J3MB,us,S,1,1,test,ColourTree 6' x 50' Black Fence Privacy Screen...,None,"★【Virgin, UV Stabilized Compounds & Thick Mate...",ColourTree,Black 2nd Generation
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2621163,2621163,ﾌﾘｸｼｮﾝ 消しゴム,130645,B00JET12SW,jp,E,1,1,test,パイロット フリクションボールメタル専用 消去用ラバー グラデーションブラック LFBTRU...,None,None,PILOT,None
2621164,2621164,ﾌﾘｸｼｮﾝ 消しゴム,130645,B00JET11BU,jp,E,1,1,test,パイロット フリクションボール専用 消去用ラバー ピンク LFBTRU10-P,None,None,PILOT,None
2621165,2621165,ﾌﾘｸｼｮﾝ 消しゴム,130645,B007M8TLZI,jp,E,1,1,test,Pilot フリクション消しゴム グレー (ELF-10-GY),None,パイロット フリクション消しゴム グレー\nグレー\nフリクションペン用消しゴム。,PILOT,グレー
2621166,2621166,ﾌﾘｸｼｮﾝ 消しゴム,130645,B004ECLQHQ,jp,I,1,1,test,パイロット 消せるカラーサインぺン フリクションカラーズ 12色 SFC-120M-12C,None,【フリクション】こすると消えるカラーペン\n【特長】こするとインキが透明に/こすっても消しカ...,PILOT,12色セット


In [13]:
test = {}
for row in df_task_1_test.iterrows():
    if row[1]['product_locale'] == 'us':
        try:
            qid = int(row[1]['query_id'])
            query_text = row[1]['query']
            label = label_map2[row[1]['esci_label']]
            if label >= 0:
                pid = row[1]['product_id']
                if qid not in test:
                    test[qid] = {}
                    test[qid]['query_text'] = query_text
                    test[qid]['labels'] = {}
                trec_id = asin2trecid[pid]
                test[qid]['labels'][trec_id] = label
        except:
            pass
print("There are {} queries in the dev postion of the dataset".format(len(test)))
print("An example query and its judgements is:\n{}".format(test[2]))

There are 8956 queries in the dev postion of the dataset
An example query and its judgements is:
{'query_text': '!qscreen fence without holes', 'labels': {529863: 0, 686755: 0, 20963: 3, 1603474: 2, 1061163: 0, 1551230: 0, 1123804: 3, 607296: 0, 959958: 3, 252024: 3, 1138651: 0, 484543: 0, 1189858: 3, 290002: 0, 326750: 0, 256723: 0, 617589: 0, 935454: 0, 1445740: 3, 528132: 0, 1334602: 3, 1450401: 0, 219550: 3, 1408517: 3, 1319046: 0, 1039500: 0, 598226: 0, 409307: 0, 1624687: 1, 1166881: 3, 549056: 2, 372111: 0, 603328: 0, 404432: 2, 938966: 0, 1273728: 0, 1045313: 0}}


In [102]:
with open('product-search-dev.qrels','w') as w:
    for qid in test:
        for doc_id in test[qid]['labels']:
            relevancy = test[qid]['labels'][doc_id]
            w.write("{}\t0\t{}\t{}\n".format(qid, doc_id, relevancy))

In [103]:
with open('product-search-dev-qrels.jsonl','w') as w:
    for qid in test:
        for doc_id in test[qid]['labels']:
            relevancy = test[qid]['labels'][doc_id]
            j = {'qid':qid, 'Q0':'Q0', 'docid':doc_id, 'rel':relevancy }
            w.write("{}\n".format(json.dumps(j)))

from pyarrow import json
table = json.read_json('product-search-dev-qrels.jsonl') 
pq.write_table(table, 'product-search-dev-qrels.parquet')
import json

In [138]:
from pyserini.search.lucene import LuceneSearcher

searcher = LuceneSearcher('indexes/esci-simple-pyserini')
with open('pyserini-simple-dev.run','w') as w:
    for qid in test:
        hits = searcher.search(test[qid]['query_text'], 1000)
        for i in range(0,len(hits)):
            w.write("{}\t0\t{}\t{}\t{}\tpyserini-simple\n".format(qid,hits[i].docid, i,hits[i].score))

In [139]:
! ir_measures ../product-Search-Qrels-v0.1/data/product-search-dev.qrels pyserini-simple-dev.run nDCG@10 P@5 'P(rel=2)@5' Judged@10 R@1000

nDCG@10	0.2134
P@5	0.2377
P(rel=2)@5	0.2216
Judged@10	0.2080
R@1000	0.5957


In [137]:
from pyserini.search.lucene import LuceneSearcher

searcher = LuceneSearcher('indexes/esci-full-pyserini')
with open('pyserini-full-dev.run','w') as w:
    for qid in test:
        hits = searcher.search(test[qid]['query_text'], 1000)
        for i in range(0,len(hits)):
            w.write("{}\t0\t{}\t{}\t{}\tpyserini-full\n".format(qid,hits[i].docid, i,hits[i].score))

In [140]:
! ir_measures ../product-Search-Qrels-v0.1/data/product-search-dev.qrels pyserini-full-dev.run nDCG@10 P@5 'P(rel=2)@5' Judged@10 R@1000

nDCG@10	0.2478
P@5	0.2720
P(rel=2)@5	0.2552
Judged@10	0.2436
R@1000	0.6631


## Since We do not have a publicly avaible test set we go ahead and construsct one using a few methods. First, we generate spearfished queries. In other words queries which have have a direct desire for an exact match title or product id. We go ahead and generate 1000

In [21]:
len(qid2query)

29844

In [15]:
new_queries = {}
new_qid = 100000 # start qid at 100000
print("starting to generate test queries")
with open('test-direct.qrel','w') as w:
    with open('test-direct-qrels.jsonl','w') as w2:
        for i in range(0,500):
            selected_document = random.randint(0, len(collection))
            rand_choice = random.randint(0,2) # 0 chooses the asin, 1 uses some exact match from title, 2 uses some exact match from text
            if rand_choice == 0:
                new_queries[new_qid] = collection[selected_document]['asin']
                candidate_query = new_queries[new_qid]
            elif rand_choice == 1:
                num_words = random.randint(3,7)
                candidate_query = ' '.join(collection[selected_document]['title'].split(' ')[:num_words])
                if len(candidate_query.split(' '))> 3:
                    new_queries[new_qid] = candidate_query
            elif rand_choice == 2:
                num_words = random.randint(3,10)
                candidate_query = ' '.join(collection[selected_document]['title'].split(' ')[:num_words])
                if len(candidate_query.split(' '))> 3:
                    new_queries[new_qid] = candidate_query
            qid2query[new_qid] = candidate_query
            doc_id = collection[selected_document]['doc_id']
            w.write("{}\t0\t{}\t1\n".format(new_qid, doc_id))
            j = {'qid':new_qid, 'Q0':'Q0', 'docid':doc_id, 'rel':1 }
            w2.write("{}\n".format(json.dumps(j)))
            new_qid += 1
        print("done generating spearfishing queries. Generating queries using LLM")
        new_qid = 200000 # generated queries start qid at 200000 for generated queries
        for i in range(0,500):
            if i % 100 == 0:
                print("{} queries generated".format(i))
            selected_document = random.randint(0, len(collection))
            prompt = 'Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n'
            prompt += '### Instruction: \n\n You are a product search engine. Given the following product title and description what would be a query which a user would issue to find this item?\n\n'
            prompt += '### Input:\n\n'
            prompt += 'Product title:{}\n'.format(collection[selected_document]['title'])
            prompt += 'Product Description:{}\n'.format(collection[selected_document]['text'])
            prompt += '### Response: \n'
            message=[{"role": "user", "content": prompt}]
            try:
                response = openai.ChatCompletion.create(
                    model="gpt-4",
                    max_tokens=100,
                    temperature=0.7,
                    messages = message)
                candidate_query = response['choices'][0]['message']['content']
                qid2query[new_qid] = candidate_query
                new_queries[new_qid]  = candidate_query
                doc_id = collection[selected_document]['doc_id']
                w.write("{}\t0\t{}\t1\n".format(new_qid, doc_id))
                j = {'qid':new_qid, 'Q0':'Q0', 'docid':doc_id, 'rel':1 }
                w2.write("{}\n".format(json.dumps(j)))
                new_qid += 1
            except:
                pass
    
print(len(qid2query)) 
print(len(new_queries))
print(new_queries)


from pyarrow import json
table = json.read_json('test-direct-qrels.jsonl') 
pq.write_table(table, 'test-direct-qrels.parquet')
import json

starting to generate test queries
done generating spearfishing queries. Generating queries using LLM
0 queries generated
100 queries generated
200 queries generated
300 queries generated
400 queries generated
30734
926
{100000: 'Nike Kids Tanjun (GS) December Sky/White Running Shoe', 100001: 'Avidlove Garter Lingerie For Women Sexy Lace Teddy', 100002: 'Lovelyshop Blue Gems Rhinestone', 100003: 'B077YKYT68', 100004: "Under Armour Men's Jungle Rat Tactical", 100005: 'qnprt Old Lady Costume', 100006: 'B073RNKSYP', 100007: 'Baseball Display Case | Baseball Holders for Balls', 100008: 'B00APRTZ1I', 100010: 'PISSION Waterproof Cases Ultra Slim Full Body Protective', 100011: 'Baby Toddler Girls Long Sleeve', 100012: 'Skater SIH3ST-A Snoopy Mix', 100015: 'Kinto 22830 ROSETTE Champagne Glass,', 100016: 'Blushing Cherry Blossom Fragrance Oil (60ml) For', 100017: 'Lion Odor Absorbing Sand, Scent Plus,', 100018: 'B015JPRSAK', 100020: 'Only Boxes, Pack 20 Cajas de Archivo, Montaje', 100021: 'Blink

In [16]:
with open('qid2query.tsv','w') as w:
    for qid in qid2query:
        w.write("{}\t{}\n".format(qid, qid2query[qid]))    

In [17]:
with open('2023-test-queries.tsv','w') as w:
    for qid in new_queries:
        w.write("{}\t{}\n".format(qid, new_queries[qid]))    

In [21]:
qid2query[qid]

'"Type A/C Type III inflatable swimming pool filter cartridge 10.6x20.3cm"'

In [22]:
from pyserini.search.lucene import LuceneSearcher

searcher = LuceneSearcher('indexes/esci-simple-pyserini')
with open('pyserini-simple-test-2023.run','w') as w:
    for qid in new_queries:
        hits = searcher.search(new_queries[qid], 100)
        for i in range(0,len(hits)):
            w.write("{}\t0\t{}\t{}\t{}\tpyserini-simple\n".format(qid,hits[i].docid, i,hits[i].score))

In [26]:
! ir_measures test-direct.qrel pyserini-simple-test-2023.run nDCG@10 P@5 'P(rel=2)@5' Judged@10 R@100

nDCG@10	0.3765
P@5	0.0850
P(rel=2)@5	0.0000
Judged@10	0.0460
R@100	0.5110


In [31]:
from pyserini.search.lucene import LuceneSearcher

searcher = LuceneSearcher('indexes/esci-full-pyserini')
with open('pyserini-full-test-2023.run','w') as w:
    for qid in new_queries:
        hits = searcher.search(qid2query[qid], 100)
        for i in range(0,len(hits)):
            w.write("{}\t0\t{}\t{}\t{}\tpyserini-full\n".format(qid,hits[i].docid, i,hits[i].score))

In [25]:
! ir_measures test-direct.qrel pyserini-full-test-2023.run nDCG@10 P@5 'P(rel=2)@5' Judged@10 R@100

nDCG@10	0.4702
P@5	0.1018
P(rel=2)@5	0.0000
Judged@10	0.0552
R@100	0.6172


In [30]:
train[qid]['query_text']

{'query_text': '!awnmower tires without rims',
 'labels': {1049092: 0,
  314519: 3,
  689593: 0,
  717649: 2,
  1397156: 3,
  1095149: 3,
  341130: 1,
  911618: 3,
  831070: 2,
  1624211: 0,
  116997: 2,
  1399124: 3,
  1035122: 3,
  932695: 0,
  580389: 0}}

In [36]:
from pyserini.search.lucene import LuceneSearcher

searcher = LuceneSearcher('indexes/esci-full-pyserini')
with open('pyserini-full-train-2023.run','w') as w:
    for qid in train:
        hits = searcher.search(train[qid]['query_text'], 100)
        for i in range(0,len(hits)):
            w.write("{}\t0\t{}\t{}\t{}\tpyserini-full\n".format(qid,hits[i].docid, i,hits[i].score))

In [35]:
from pyserini.search.lucene import LuceneSearcher

searcher = LuceneSearcher('indexes/esci-simple-pyserini')
with open('pyserini-simple-train-2023.run','w') as w:
    for qid in train:
        hits = searcher.search(train[qid]['query_text'], 100)
        for i in range(0,len(hits)):
            w.write("{}\t0\t{}\t{}\t{}\tpyserini-simple\n".format(qid,hits[i].docid, i,hits[i].score))

## Now we can go ahead and generate triples. We will use positive labels we have and mine negatives from bm25 top 100

In [49]:
for doc in train[qid]['labels']:
    print(doc)

1049092
314519
689593
717649
1397156
1095149
341130
911618
831070
1624211
116997
1399124
1035122
932695
580389


In [51]:
collection[1095149]['description']

'From the brand Previous page MaxAuto is a thriving brand which has been committed to meeting customer needs with high-quality tires. Our lineup include tires applied in Lawn & Garden, Agriculture, Golf Cart, ATV/UTV, ST Trailer, light truck, etc. We\'ve obtained such certification as US DOT(2000),EU ECE(2006),Middle East SASO. The testing center has passed the CNAS(2017), German TÜV(2018), to ensure that all of our products are manufactured with the same quality standards. Lawn Mower Tires Visit the Store Lawn Mower Tires Visit the Store Lawn Mower Tires Visit the Store Lawn Mower Tires Visit the Store Lawn Mower Tires Visit the Store Next page Product Description About MaxAuto Brand >>Focus on high-quality speciality tire manufacturer In 1997, we began to invest in the development and production of ATV tires. Since then, we continue to expand our technical capabilities, production capacity and research and development. With a comprehensive range of applications and broad product line

In [59]:
collection[int(hits[0].docid)]

{'type': 'product',
 'locale': 'us',
 'asin': 'B082K7V2GZ',
 'title': 'NEIKO 20601A 14.5” Steel Tire Spoons Tool Set, Tire Tools Include 3 Piece Tire Spoons, 3 Piece Rim Protector, Valve Tool, 6 Piece Valve Cores, Motorcycle Tire Changer, Dirt Bike Tire Levers',
 'stars': '4.5 out of 5 stars',
 'ratings': '5,926 ratings',
 'category': ['Automotive',
  'Tools & Equipment',
  'Tire & Wheel Tools',
  'Tire Repair Tools'],
 'attrs': {},
 'bullets': ['Make sure this fits by entering your model number.',
  'STEEL-IRON TIRE SPOONS: Each tire lever is constructed of hardened steel-iron with curved tips that provides strong toughness and reliable durability.',
  'WHEEL PROTECTION: Each tire spoon kit comes with three highly durable polypropylene rim protectors to prevent damage to the rims while working on motorcycle, lawnmower, or dirt bike in conjunction with the tire changing tools.',
  'LONGER TIRE TOOLS: Each tire iron spoon is an extra-long lever at 14.5 inches and offers more leverage an

In [72]:
num_negatives = 100
num_written = 0
with open("train.jsonl",'w') as w:
    for qid in train:
        if num_written % 100 == 0:
            print("{} samples processed".format(num_written))
        j = {}
        j['query_id'] = qid
        j['query'] = train[qid]['query_text']
        j['positive_passages'] = []
        j['negative_passages'] = []
        prove_positive = []
        for doc_id in train[qid]['labels']:
            relevance = train[qid]['labels'][doc_id]
            if relevance == 3:
                doc = collection[doc_id]
            i = {}
            prove_positive.append(doc_id) # keep track of positives to ensure we don't count them as random negatives
            i['docid'] = doc_id
            i['title'] = collection[doc_id]['title']
            i['text'] = collection[doc_id]['description']
            j['positive_passages'].append(i)
        hits = searcher.search(j['query'], num_negatives)
        for z in range(0,len(hits)):
            if hits[z].docid not in prove_positive:
                i = {}
                i['docid'] = int(hits[z].docid)
                i['title'] = collection[i['docid']]['title']
                i['text'] = collection[i['docid']]['description']
                j['negative_passages'].append(i)
        w.write('{}\n'.format(json.dumps(j)))
        num_written += 1
        
#from pyarrow import json
#table = json.read_json('train.jsonl') 
#pq.write_table(table, 'train.parquet')
#import json
from json2parquet import convert_json
convert_json('train.jsonl','train.parquet') # have to do this as the block size has some issue 

0 samples processed
100 samples processed
200 samples processed
300 samples processed
400 samples processed
500 samples processed
600 samples processed
700 samples processed
800 samples processed
900 samples processed
1000 samples processed
1100 samples processed
1200 samples processed
1300 samples processed
1400 samples processed
1500 samples processed
1600 samples processed
1700 samples processed
1800 samples processed
1900 samples processed
2000 samples processed
2100 samples processed
2200 samples processed
2300 samples processed
2400 samples processed
2500 samples processed
2600 samples processed
2700 samples processed
2800 samples processed
2900 samples processed
3000 samples processed
3100 samples processed
3200 samples processed
3300 samples processed
3400 samples processed
3500 samples processed
3600 samples processed
3700 samples processed
3800 samples processed
3900 samples processed
4000 samples processed
4100 samples processed
4200 samples processed
4300 samples processed


In [79]:
num_negatives = 100
with open("dev.jsonl",'w') as w:
    for qid in test:
        j = {}
        j['query_id'] = qid
        j['query'] = test[qid]['query_text']
        j['positive_passages'] = []
        j['negative_passages'] = []
        prove_positive = []
        for doc_id in test[qid]['labels']:
            relevance = test[qid]['labels'][doc_id]
            if relevance == 3:
                doc = collection[doc_id]
            i = {}
            prove_positive.append(doc_id) # keep track of positives to ensure we don't count them as random negatives
            i['docid'] = doc_id
            i['title'] = collection[doc_id]['title']
            i['text'] = collection[doc_id]['description']
            j['positive_passages'].append(i)
        hits = searcher.search(j['query'], num_negatives)
        for z in range(0,len(hits)):
            if hits[z].docid not in prove_positive:
                i = {}
                i['docid'] = int(hits[z].docid)
                i['title'] = collection[i['docid']]['title']
                i['text'] = collection[i['docid']]['description']
                j['negative_passages'].append(i)
        w.write('{}\n'.format(json.dumps(j)))

from pyarrow import json
table = json.read_json('dev.jsonl') 
pq.write_table(table, 'dev.parquet')
import json

In [80]:
num_negatives = 100
with open("test.jsonl",'w') as w:
    for qid in new_queries:
        j = {}
        j['query_id'] = qid
        j['query'] = new_queries[qid]
        j['positive_passages'] = []
        j['negative_passages'] = []
        w.write('{}\n'.format(json.dumps(j)))

from pyarrow import json
table = json.read_json('test.jsonl') 
pq.write_table(table, 'test.parquet')
import json